In [1]:
import pandas as pd
pd.set_option('display.max_rows', 4)
import numpy as np
import os
import pickle

path = 'data/statistics/'
files = os.listdir(path)
files

['oil_Jan-01-2016_Aug-17-2018.csv',
 'nikkei_Jan-01-2016_Aug-17-2018.csv',
 'ffa_Jan-01-2016_Aug-17-2018.csv',
 'bci_Jan-01-2016_Aug-17-2018.csv',
 'korea_Jan-01-2016_Aug-17-2018.csv',
 'pick_Jan-01-2016_Aug-17-2018.csv',
 'iyt_Jan-01-2016_Aug-17-2018.csv',
 'usdollar_Jan-01-2016_Aug-17-2018.csv',
 'coal_Jan-01-2016_Aug-17-2018.csv',
 'sp500_Jan-01-2016_Aug-17-2018.csv',
 'shanghai_Jan-01-2016_Aug-17-2018.csv',
 'germany_Jan-01-2016_Aug-17-2018.csv',
 'ironore_Jan-01-2016_Aug-17-2018.csv',
 'steelstock_Jan-01-2016_Aug-17-2018.csv']

In [2]:
x_df = []
y_df = None

for file in files:
    df = pd.read_csv(path+file)
#     df = pd.read_csv(path+file, dtype={'date':str, 'price':float})
    df.columns = ['date', file.split("_")[0]]
    if not file=='bci_Jan-01-2016_Aug-17-2018.csv':
        print('x:', path+file)
        x_df.append(df)
    else:
        print('y:', path+file)
        y_df = df
    if not df.iloc[:,1].dtype == 'float64':
        print("[W] please check data type. df.iloc[:,1].dtype ==", df.iloc[:,1].dtype)

print("\ntrain:", y_df.iloc[0]['date'], "~", y_df.iloc[860]['date'])
print("train:", y_df.iloc[861]['date'], "~", y_df.iloc[959]['date'])
y_df

x: data/statistics/oil_Jan-01-2016_Aug-17-2018.csv
x: data/statistics/nikkei_Jan-01-2016_Aug-17-2018.csv
x: data/statistics/ffa_Jan-01-2016_Aug-17-2018.csv
[W] please check data type. df.iloc[:,1].dtype == int64
y: data/statistics/bci_Jan-01-2016_Aug-17-2018.csv
[W] please check data type. df.iloc[:,1].dtype == object
x: data/statistics/korea_Jan-01-2016_Aug-17-2018.csv
x: data/statistics/pick_Jan-01-2016_Aug-17-2018.csv
x: data/statistics/iyt_Jan-01-2016_Aug-17-2018.csv
x: data/statistics/usdollar_Jan-01-2016_Aug-17-2018.csv
x: data/statistics/coal_Jan-01-2016_Aug-17-2018.csv
x: data/statistics/sp500_Jan-01-2016_Aug-17-2018.csv
x: data/statistics/shanghai_Jan-01-2016_Aug-17-2018.csv
x: data/statistics/germany_Jan-01-2016_Aug-17-2018.csv
x: data/statistics/ironore_Jan-01-2016_Aug-17-2018.csv
x: data/statistics/steelstock_Jan-01-2016_Aug-17-2018.csv

train: Jan 01, 2016 ~ May 10, 2018
train: May 11, 2018 ~ Aug 17, 2018


,date,bci
0,"Jan 01, 2016",472.0
1,"Jan 02, 2016",472.0
...,...,...
958,"Aug 16, 2018","3,476.00"
959,"Aug 17, 2018","3,454.00"


In [3]:
print("before:", y_df.iloc[:,1].dtype)
y_df.iloc[:,1] = y_df.iloc[:,1].map(lambda x: float(str(x).replace(',', '')))
print("after :", y_df.iloc[:,1].dtype)

before: object
after : float64


In [4]:
x_df = pd.concat([df.drop(columns='date') for df in x_df], axis=1)
x_df

,oil,nikkei,ffa,korea,pick,iyt,usdollar,coal,sp500,shanghai,germany,ironore,steelstock
0,36.76,18818.58008,4150,1911.930054,16.52,132.320007,98.97,38.05,2038.199951,3296.26,10485.80957,41.95,206.66
1,36.76,18818.58008,4150,1911.930054,16.52,132.320007,98.97,38.05,2038.199951,3296.26,10485.80957,41.95,206.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...
958,65.46,21980.82031,25250,2233.050049,30.08,201.100006,96.54,65.40,2831.439941,2705.19,12202.12988,67.59,210.85
959,65.91,22313.18945,25250,2244.610107,30.39,201.100006,95.98,65.40,2831.439941,2668.97,12202.12988,67.99,213.69


In [5]:
X = np.array(x_df)
print(X)
Y = np.array(y_df.drop(columns='date'), np.float).reshape(-1, 1)
print(Y[:6])
print(X.shape, Y.shape)

[[   36.76    18818.58008  4150.      ... 10485.80957    41.95
    206.66   ]
 [   36.76    18818.58008  4150.      ... 10485.80957    41.95
    206.66   ]
 [   36.76    18818.58008  4150.      ... 10485.80957    41.95
    206.66   ]
 ...
 [   65.01    22368.11914 25250.      ... 12400.26953    67.72
    213.1    ]
 [   65.46    21980.82031 25250.      ... 12202.12988    67.59
    210.85   ]
 [   65.91    22313.18945 25250.      ... 12202.12988    67.99
    213.69   ]]
[[472.]
 [472.]
 [472.]
 [472.]
 [462.]
 [467.]]
(960, 13) (960, 1)


In [6]:
x_train, x_test = X[:861], X[861:]
y_train, y_test = Y[:861], Y[861:]
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(861, 13) (861, 1)
(99, 13) (99, 1)


In [7]:
with open("BalticDryIndexDataset.npy", 'wb') as f:
    pickle.dump(((x_train, y_train), (x_test, y_test)), f)
with open("BalticDryIndexDataset.npy", 'rb') as f:
    (x_train, y_train), (x_test, y_test) = pickle.load(f)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(861, 13) (861, 1)
(99, 13) (99, 1)
